In [19]:
import cv2 
import mediapipe as mp 
import numpy as np 
import pandas as pd

In [20]:
#Build Keypoints using MP Holistic 
mp_holistic = mp.solutions.holistic # Holistic model 
mp_drawing = mp.solutions.drawing_utils # Drawing utilities 

In [21]:
def mediapipe_detection(image, model): 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB 
#     image.flags.writable = False                  # Image is no longer writable 
    results = model.process(image)                 # Make prediction 
#     image.flags.writable = True                   # Image is now writable  
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR CONVERSION RGB 2 BGR 
    return image, results 
    
def draw_landmarks(image, results): 
#     mp_drawing.draw_landmarks( 
#       image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections 
#     mp_drawing.draw_landmarks( 
#       image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections 
    mp_drawing.draw_landmarks( 
      image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections 
    mp_drawing.draw_landmarks( 
      image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections 
      
def draw_styled_landmarks(image, results): 
    # Draw face connections 
#     mp_drawing.draw_landmarks( 
#       image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
#       mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),  
#       mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1))  
#     # Draw pose connections 
#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
#                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),  
#                              mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2) 
#                              )  
    # Draw left hand connections 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,  
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),  
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2) 
                             )  
    # Draw right hand connections   
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,  
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),  
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                             )  


In [22]:
# #Main function 
# cap = cv2.VideoCapture(0) 
# # Set mediapipe model  
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 
#     while cap.isOpened(): 
  
#         # Read feed 
#         ret, frame = cap.read() 
  
#         # Make detections 
#         image, results = mediapipe_detection(frame, holistic) 
#         print(results) 
          
#         # Draw landmarks 
#         draw_styled_landmarks(image, results) 
  
#         # Show to screen 
#         cv2.imshow('OpenCV Feed', image) 
  
#         # Break gracefully 
#         if cv2.waitKey(10) & 0xFF == ord('q'): 
#             break
#     cap.release() 
#     cv2.destroyAllWindows()

In [23]:
# # This operates on last results before I close the webcam feed 
# # So if there was no hand at the end (which obviously wouldnt be otherwise how would I click q)
# # then there would be 0s 

# zero_subs = [np.array([0, 0]) for _ in range(21)]

# lh = (np.array([[res.x, res.y] for res in results.left_hand_landmarks.landmark])) if results.left_hand_landmarks else zero_subs

# rh = (np.array([[res.x, res.y] for res in results.right_hand_landmarks.landmark])) if results.right_hand_landmarks else zero_subs


In [24]:
# # Mediapipe returns normalized coordinates , 
# # But to feed to a ML model, coordinates won't matter, only relative positions will
# # So taking difference wrt 1st coordinate pair 

# print(rh)
# print(lh)


# lh = [(lh[i]-lh[0]).tolist() for i in range(1,len(lh))]
# lh.insert(0, [0, 0])
# print(lh)

# rh = [(rh[i]-rh[0]).tolist() for i in range(1,len(rh))]
# rh.insert(0, [0, 0])
# print(rh)

In [25]:
# lh = np.array(lh)
# rh = np.array(rh)
# lh.flatten()
# rh.flatten()

In [26]:
data_A = pd.DataFrame()
data_Y = pd.DataFrame()
data_U = pd.DataFrame()
data_S = pd.DataFrame()
data_H = pd.DataFrame()

classes = ['A','Y','U','S','H']

In [50]:
# Rerun this piece of code manually with changing A, Y, U ... to collect the dataset as pandas dataframe 
# search for ASL sign language alphabets to see what gesture to make manually 

#Main function 
cap = cv2.VideoCapture(0) 
# Set mediapipe model  
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 
    while cap.isOpened(): 
  
        # Read feed 
        ret, frame = cap.read() 
        
        frame = cv2.flip(frame,1)
        
        # Make detections 
        image, results = mediapipe_detection(frame, holistic) 
        print(results) 
          
        # Draw landmarks 
        draw_styled_landmarks(image, results) 
  
        # Show to screen 
        cv2.imshow('OpenCV Feed', image) 
          
        zero_subs = [np.array([0, 0]) for _ in range(21)]
        
        # As I have mirrored the camera frame , left_hand_landmarks contains information for right hand
        # I will only use right hand ASL sign language 
        rh = (np.array([[res.x, res.y] for res in results.left_hand_landmarks.landmark])) if results.left_hand_landmarks else zero_subs
        
        rh = [(rh[i]-rh[0]).tolist() for i in range(1,len(rh))]
        if(rh!=zero_subs):
            rh.insert(0, [0, 0])
        
        rh = np.array(rh)
        rh = rh.flatten()
        
        print(rh)
        
        if not np.all(rh == 0):
            data_S = data_S._append(pd.Series(rh), ignore_index=True)
        
        # Break gracefully 
        if cv2.waitKey(10) & 0xFF == ord('q'): 
            break
    cap.release() 
    cv2.destroyAllWindows()

C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.          0.         -0.05172181 -0.06555331 -0.08766031 -0.14672184
 -0.07190728 -0.23277864 -0.02981687 -0.27923742 -0.09338695 -0.21788538
 -0.10382801 -0.2932727  -0.09077156 -0.21537596 -0.07868075 -0.1860882
 -0.04978591 -0.22490385 -0.05679417 -0.3007223  -0.05064583 -0.22129229
 -0.04517609 -0.2022537  -0.00646883 -0.22132581 -0.00969875 -0.28864181
 -0.01178157 -0.213067   -0.00969279 -0.19070053  0.03503114 -0.20919868
  0.02861339 -0.26391506  0.02169353 -0.21344376  0.02260715 -0.18724698]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.          0.         -0.05263811 -0.05129457 -0.08960867 -0.12943518
 -0.07395828 -0.20646024 -0.03292155 -0.24633077 -0.09757262 -0.20283332
 -0.10622597 -0.27074522 -0.09003901 -0.20189539 -0.08042896 -0.17280269
 -0.05430877 -0.21301284 -0.05645335 -0.27785784 -0.04804862 -0.20361033
 -0.04701608 -0.18399143 -0.01049387 -0.21236718 -0.01023549 -0.26673555
 -0.009802

C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.          0.         -0.05307597 -0.04652452 -0.08689928 -0.11973619
 -0.07372367 -0.19335291 -0.03945804 -0.2349501  -0.09403002 -0.19514769
 -0.10262823 -0.26133776 -0.08591986 -0.20628798 -0.07579321 -0.17486507
 -0.05162805 -0.20714286 -0.05482638 -0.27130392 -0.04465216 -0.20746309
 -0.04346234 -0.18497875 -0.00862843 -0.20744035 -0.0094381  -0.25985467
 -0.00784057 -0.19743323 -0.01243359 -0.17812234  0.03301871 -0.19705814
  0.03012353 -0.23929259  0.02647609 -0.19127992  0.02239555 -0.16866028]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.          0.         -0.051018   -0.04487705 -0.08727229 -0.12282705
 -0.0704813  -0.19667605 -0.0293144  -0.23311865 -0.09603012 -0.19818407
 -0.10352582 -0.26183841 -0.08590198 -0.19821325 -0.07574451 -0.17025226
 -0.05319816 -0.2093918  -0.05513245 -0.26947749 -0.04510832 -0.19634271
 -0.04361135 -0.17903277 -0.00950986 -0.20798877 -0.00952864 -0.25888032
 -0.00795

C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.          0.         -0.06238848 -0.05193049 -0.10479242 -0.14370304
 -0.08665246 -0.22669399 -0.03992134 -0.25490245 -0.09148383 -0.20365581
 -0.11216146 -0.27375636 -0.09944636 -0.20355916 -0.0846833  -0.17196387
 -0.05249649 -0.21463862 -0.06779736 -0.28629613 -0.06145024 -0.20722732
 -0.05001312 -0.18270975 -0.01144421 -0.21566072 -0.02443057 -0.27791819
 -0.02464044 -0.20118272 -0.01628083 -0.17617917  0.03206527 -0.20922953
  0.01506382 -0.26414844  0.00929666 -0.21285707  0.01650888 -0.18689656]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.          0.         -0.06226057 -0.05225587 -0.1044023  -0.14175063
 -0.08740789 -0.22521162 -0.04078406 -0.25204962 -0.09116918 -0.20166862
 -0.11187005 -0.27388757 -0.09840286 -0.20514357 -0.08352721 -0.17198378
 -0.05195558 -0.21259871 -0.06739217 -0.2864286  -0.06064129 -0.21019211
 -0.05016387 -0.18392283 -0.01089877 -0.21350199 -0.02410597 -0.27930433
 -0.02408

C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.          0.         -0.06383562 -0.04698217 -0.10682631 -0.14040285
 -0.08622867 -0.22699887 -0.03945899 -0.2510972  -0.09388053 -0.20091072
 -0.11479497 -0.27527571 -0.10163814 -0.20897514 -0.08681393 -0.17334515
 -0.05382872 -0.21334064 -0.06908619 -0.28734681 -0.06207812 -0.21249744
 -0.05243939 -0.18554407 -0.0124684  -0.21478561 -0.02523905 -0.28238201
 -0.02477455 -0.20767748 -0.01837355 -0.1799196   0.03089046 -0.20824125
  0.01527858 -0.26307261  0.00972414 -0.2123948   0.01568931 -0.18607271]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.          0.         -0.06278777 -0.05099869 -0.10311282 -0.13954586
 -0.08649832 -0.2212075  -0.04303253 -0.25458118 -0.09032893 -0.20202142
 -0.1108796  -0.27440831 -0.09929305 -0.2098006  -0.08356506 -0.1739803
 -0.05145788 -0.21235347 -0.06618834 -0.28651255 -0.06140536 -0.21384215
 -0.05058563 -0.18286783 -0.01034617 -0.21379018 -0.02319562 -0.28022447
 -0.024669

C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\anaconda_installed_22_12_23\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.          0.         -0.06315416 -0.0457688  -0.10623181 -0.14037669
 -0.08653122 -0.22796631 -0.03916323 -0.25187388 -0.09323955 -0.1991508
 -0.11320418 -0.27445981 -0.10033268 -0.21185333 -0.08632118 -0.17685056
 -0.05284363 -0.21241182 -0.06752843 -0.28886592 -0.06142306 -0.21904317
 -0.052113   -0.19074404 -0.01128167 -0.21396285 -0.0233199  -0.28440505
 -0.02375555 -0.21411926 -0.01745653 -0.18462354  0.03254831 -0.20746544
  0.01737362 -0.26359975  0.01059151 -0.21582511  0.01582634 -0.18876159]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
[ 0.          0.         -0.06346077 -0.04860085 -0.106197   -0.14056301
 -0.08691674 -0.22642475 -0.03923911 -0.25436032 -0.09162229 -0.20165601
 -0.11335868 -0.27670425 -0.0995025  -0.2089861  -0.08383811 -0.17392105
 -0.05169779 -0.21340168 -0.06727761 -0.28833306 -0.06076008 -0.21275222
 -0.05075234 -0.18549079 -0.01047415 -0.21491376 -0.02353364 -0.28107184
 -0.023848

In [53]:
# Just adding a class tag which will be the target class later while predicting 

data_A['class'] = 0
data_Y['class'] = 1
data_U['class'] = 2
data_S['class'] = 3
data_H['class'] = 4

# print(data_A)
print()

In [55]:
print(data_A.head())
print(len(data_A))
print(len(data_Y))
print(len(data_U))
print(len(data_S))
print(len(data_H))

   class    0    1         2         3         4         5         6  \
0      0  0.0  0.0 -0.083955 -0.057687 -0.143967 -0.166604 -0.158132   
1      0  0.0  0.0 -0.083900 -0.061100 -0.141032 -0.165363 -0.158884   
2      0  0.0  0.0 -0.086185 -0.057912 -0.151771 -0.144644 -0.180921   
3      0  0.0  0.0 -0.084918 -0.037873 -0.150861 -0.124648 -0.172609   
4      0  0.0  0.0 -0.080348 -0.028174 -0.146617 -0.121466 -0.170870   

          7         8  ...        32        33        34        35        36  \
0 -0.270313 -0.146678  ...  0.008286 -0.224137  0.067247 -0.265167  0.064527   
1 -0.270144 -0.152182  ...  0.000033 -0.222951  0.056077 -0.252391  0.053019   
2 -0.243660 -0.172599  ... -0.028672 -0.220826  0.027602 -0.257759  0.024671   
3 -0.227299 -0.169301  ... -0.027564 -0.208170  0.024663 -0.242198  0.024185   
4 -0.225190 -0.166369  ... -0.027546 -0.204892  0.024202 -0.238494  0.024323   

         37        38        39        40        41  
0 -0.321461  0.054204 -0.252515 

In [56]:
# Now combining the mini datasets I have gathered for the alphabets 
# I took 200 enteries for all the alphabets to ensure class balance 

dflist = [data_A.iloc[0:200],data_Y.iloc[0:200],data_U.iloc[0:200],data_S.iloc[0:200],data_H.iloc[0:200]]

df = pd.concat(dflist, ignore_index=True)

print(df.head())
print(len(df))

   class    0    1         2         3         4         5         6  \
0      0  0.0  0.0 -0.083955 -0.057687 -0.143967 -0.166604 -0.158132   
1      0  0.0  0.0 -0.083900 -0.061100 -0.141032 -0.165363 -0.158884   
2      0  0.0  0.0 -0.086185 -0.057912 -0.151771 -0.144644 -0.180921   
3      0  0.0  0.0 -0.084918 -0.037873 -0.150861 -0.124648 -0.172609   
4      0  0.0  0.0 -0.080348 -0.028174 -0.146617 -0.121466 -0.170870   

          7         8  ...        32        33        34        35        36  \
0 -0.270313 -0.146678  ...  0.008286 -0.224137  0.067247 -0.265167  0.064527   
1 -0.270144 -0.152182  ...  0.000033 -0.222951  0.056077 -0.252391  0.053019   
2 -0.243660 -0.172599  ... -0.028672 -0.220826  0.027602 -0.257759  0.024671   
3 -0.227299 -0.169301  ... -0.027564 -0.208170  0.024663 -0.242198  0.024185   
4 -0.225190 -0.166369  ... -0.027546 -0.204892  0.024202 -0.238494  0.024323   

         37        38        39        40        41  
0 -0.321461  0.054204 -0.252515 

In [57]:
df.to_excel("ASL_alphabets.xlsx") 